In [131]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from transformers import BertTokenizerFast, TFBertModel
import math
from tqdm.notebook import tqdm

In [76]:
billboard_text = pd.read_csv("data/billboard-lyrics.csv")
billboard_text["billboard"] = True
billboard_text.head()

,Unnamed: 0,Name,Lyrics,billboard
0,0,Easy On Me,\nThere ain't no gold in this river\nThat I've...,True
1,1,Stay,\nI do the same thing I told you that I never ...,True
2,2,Industry Baby,"\n(D-D-Daytrip took it to ten, hey)\nBaby back...",True
3,3,Fancy Like,"\nAyy\nMy girl is bangin', she's so low mainte...",True
4,4,Bad Habits,"\n(One, two, three, four)\nOoh, ooh\n\n\nEvery...",True


In [77]:
non_billboard_text = pd.read_csv("data/non-billboard-lyrics.csv")
non_billboard_text["billboard"] = False
non_billboard_text.head()

,Unnamed: 0,Name,Lyrics,billboard
0,0,I'll Love You Next Life,"Try, try, try\n試一試\nBaby, don’t you cry\n寶貝別哭了...",False
1,1,Step Up,"\n(Step, step, step up)\nThis is something for...",False
2,2,Just Like Me,\nLilRiico got the sauce\n\n\nΔεν κοιμάται μον...,False
3,3,Marly-Gomont,\n\n\nDédicacé à tous ceux qui viennent des p'...,False
4,4,Ham kummst,\nLetzte Nocht wor a schware Partie fia mi\nDa...,False


In [81]:
text = pd.concat([billboard_text, non_billboard_text])
text.sample(5)

,Unnamed: 0,Name,Lyrics,billboard
7149,7149,Kaksi Onnellista,Missä on se pieni tyttö\nJoka nauravin silmin\...,False
10265,10265,Something's Always Wrong,"\nAnother day, I call and never speak\nAnd you...",True
4590,4590,The One That Got Away,"\nSummer after high school, when we first met\...",True
2382,2382,Wanted You,\nWhere you at?\nAt my house\nLet me see that ...,True
1947,1947,Mess,\nBecause my days is a mess\nMy nights is a me...,True


In [128]:
tokenizer = BertTokenizerFast.from_pretrained('bert-large-uncased')
bert_model = TFBertModel.from_pretrained("bert-large-uncased")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Downloading:   0%|          | 0.00/1.37G [00:00<?, ?B/s]

Some layers from the model checkpoint at bert-large-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-large-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [83]:
lyrics_list = text["Lyrics"].dropna().astype(str).tolist()


In [ ]:
batch_size = 32
batches = math.ceil(len(lyrics_list) / batch_size)
acc = list()

for x in tqdm(range(batches)):
    vectors = tokenizer(lyrics_list[batch_size * x:batch_size * (x + 1)], truncation=True, padding=True, return_tensors="tf")
    output = bert_model(**vectors)
    acc.append(output.pooler_output)

acc = tf.stack(acc, axis=0)

  0%|          | 0/654 [00:00<?, ?it/s]

In [126]:
model(**vectors)

ValueError: The first argument to `Layer.call` must always be passed.

In [105]:
text["output"] = text["billboard"].apply(lambda val: 1 if val else 0)

In [113]:
model = keras.Sequential([
    keras.layers.InputLayer(input_shape=(512,)),
    # keras.layers.Dense(512, activation=keras.activations.relu),
    # keras.layers.Dense(512, activation=keras.activations.relu),
    # keras.layers.Dense(512, activation=keras.activations.relu),
    keras.layers.Dense(2),
])

model.compile(optimizer='adam',
              loss=tf.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'],
              )

In [117]:
model.fit(acc,
          text["output"],
          epochs=2,
          use_multiprocessing=True
          )

Epoch 1/2


ValueError: in user code:

    File "/Users/prathamrawat/git/BillBERT/venv/lib/python3.8/site-packages/keras/engine/training.py", line 878, in train_function  *
        return step_function(self, iterator)
    File "/Users/prathamrawat/git/BillBERT/venv/lib/python3.8/site-packages/keras/engine/training.py", line 867, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/prathamrawat/git/BillBERT/venv/lib/python3.8/site-packages/keras/engine/training.py", line 860, in run_step  **
        outputs = model.train_step(data)
    File "/Users/prathamrawat/git/BillBERT/venv/lib/python3.8/site-packages/keras/engine/training.py", line 809, in train_step
        loss = self.compiled_loss(
    File "/Users/prathamrawat/git/BillBERT/venv/lib/python3.8/site-packages/keras/engine/compile_utils.py", line 201, in __call__
        loss_value = loss_obj(y_t, y_p, sample_weight=sw)
    File "/Users/prathamrawat/git/BillBERT/venv/lib/python3.8/site-p